In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import datetime 
import schedule 
import smtplib
import sqlite3
import sqlalchemy
import re

con=sqlite3.connect('model_stock.db')
engine=sqlalchemy.create_engine('sqlite:///model_stock.db')
cur=con.cursor()
cur.execute('CREATE TABLE NepalStock1.3 (Num,Symbol,LTP,Change,High,Low,Open,Qty,Turnover)')


def extracting_data():

        url="https://merolagani.com/StockQuote.aspx"
        headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36" 
         }
        page = requests.get(url)
        soup1=BeautifulSoup(page.text,'lxml')
        #soup2=BeautifulSoup(soup1.prettify(),"html.parser")

        #lists=soup1.find_all('td',class_="text-right").text

        #lists= soup1.find_all('td', attrs = {'class':"text-right"}).get_text

        #print(lists)
        lists2=soup1.find('table',class_='table table-bordered table-striped table-hover sortable')
        #print(lists2)
        headers=[]
        for i in lists2.find_all('th'): 

                title=i.text
                headers.append(title)

        mydata = pd.DataFrame(columns=['Num','Symbol','LTP','Change','High','Low','Open','Qty','Turnover'])

        for j in lists2.find_all('tr')[1:]:

                row_data = j.find_all('td')
                row = [i.text for i in row_data]
                length = len(mydata)
                mydata.loc[length] = row

        mydata.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
        # Drop “#” column
        #mydata.drop('\n#', inplace=True, axis=1)
        # Export to csv
        mydata.to_csv('dataset_stock.csv', index=False)
        # Try to read csv
        mydata2 = pd.read_csv('dataset_stock.csv')            
        # print(mydata2)      
       
      
        mydata2.to_sql('NepalStock1.3',engine,if_exists='append',index=False)

        cur.execute("select * from NepalStock2")

        for record in cur:
            print(record)
        #con.commit()
        #cursor.close()
        #con.close()    

#we specify the name of the function to run every 3600 seconds to save data 'datase_stock.csv' 
#in the directory then in the model in sql python

# to show results we consider 10 seconds int this example
schedule.every(10).seconds.do(extracting_data)

#we need to use a loop to make sure this function to run

while 1:
    schedule.run_pending()
    time.sleep(1)
        


OperationalError: near ".3": syntax error